In [1]:
!pip3 install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 29.3 MB/s eta 0:00:00


In [2]:
!pip3 install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=1211546 sha256=29a8f385ff51fc413323fea5b222764f6019dfe7254636811c06a48b33d6c90d
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [3]:
#Import required libraries
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

import scipy.sparse
from scipy.sparse import csr_matrix
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

# Q1

In [4]:
## Reading CSVs as dataframe

df_phone_review_1 = pd.read_csv("phone_user_review_file_1.csv", encoding='latin-1')
df_phone_review_2 = pd.read_csv("phone_user_review_file_2.csv", encoding='latin-1')
df_phone_review_3 = pd.read_csv("phone_user_review_file_3.csv", encoding='latin-1')
df_phone_review_4 = pd.read_csv("phone_user_review_file_4.csv", encoding='latin-1')
df_phone_review_5 = pd.read_csv("phone_user_review_file_5.csv", encoding='latin-1')
df_phone_review_6 = pd.read_csv("phone_user_review_file_6.csv", encoding='latin-1')

In [5]:
df_phone_review_1.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [6]:
df_phone_review_2.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,"The telephone headset is of poor quality , not...",luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,This is my first smartphone so I have nothing ...,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,Great phone. Battery life not great but seems ...,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Best 90 quid I've ever spent on a smart phone,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,I m happy with this phone.it s very good.thx team,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...


In [7]:
df_phone_review_3.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,"recomendo, eu comprei um, a um ano, e agora co...",herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Comprei um pouco desconfiada do site e do celu...,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,"Muito bom o produto, obvio que tem versÃµes me...",Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Unica ressalva fica para a camera que poderia ...,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ..."


In [8]:
df_phone_review_4.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,was not conpatable with my phone as stated. I ...,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Decent Functions and Easy to Operate Pros:- Th...,Expert Review,Samsung Galaxy Star Pro S7262 Black
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Not Good Phone such price. Hang too much and v...,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,not bad for features,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,Excellent product,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)


In [9]:
df_phone_review_5.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


In [10]:
df_phone_review_6.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,I've had the phone for awhile and it's a prett...,ajabrams95,Samsung Instinct HD
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,to be clear it is not the sellers fault that t...,Stephanie,Samsung SPH M800 Instinct
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,Well i love this phone. i have had ton of phon...,snickers,Instinct M800
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,I have had my Instinct for several months now ...,A4C,Samsung Instinct
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,i have had this instinct phone for about two m...,betaBgood,Samsung Instinct


## A. Merge all the provided CSVs into one dataFrame

In [11]:
## Since all CSVs as same column names, we'll do a concat (or union) of the 6 csvs in 1 dataframe
df_phone_review = pd.concat([df_phone_review_1, df_phone_review_2, df_phone_review_3, df_phone_review_4, df_phone_review_5, df_phone_review_6])

In [12]:
df_phone_review.shape

(1415133, 11)

## B. Explore, understand the Data and share at least 2 observations

In [13]:
## Statistical Summary of the data
df_phone_review.describe()

,score,score_max
count,1.351644e+06,1351644.0
mean,8.007060e+00,10.0
std,2.616121e+00,0.0
min,2.000000e-01,10.0
25%,7.200000e+00,10.0
50%,9.200000e+00,10.0
75%,1.000000e+01,10.0
max,1.000000e+01,10.0


In [14]:
## Checking for DataTypes for all the columns
df_phone_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


In [15]:
## Checking for null for all the columns
df_phone_review.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [16]:
## Checking for Duplicate Rows
df_phone_review[df_phone_review.duplicated() == True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
342,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
343,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
371,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,9.6,10.0,Tot nu toe in de eerste dagen weinig tot geen ...,Robin Winckens,Samsung Galaxy S8 Zwart
452,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,Wat een huge scherm!,B Yapar,Samsung Galaxy S8 Zwart
3718,/cellphones/samsung-galaxy-s6-edgeplus/,10/4/2015,en,us,Amazon,amazon.com,10.0,10.0,"For readers, my opinions on this device (ATT 3...",FrozenCloud,"Samsung Galaxy S6 Edge+, Black 64GB (Verizon W..."
...,...,...,...,...,...,...,...,...,...,...,...
146836,/cellphones/nokia-8850/,8/31/2010,ru,ru,Yandex,market.yandex.ru,10.0,10.0,"ÐÑÐ¾ÑÑÐ¾ Ð¾ÑÐ»Ð¸ÑÐ½ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½, ...",NaN,Nokia 8850
146838,/cellphones/nokia-8850/,6/25/2010,ru,ru,Yandex,market.yandex.ru,10.0,10.0,"ÐÐ° ÑÐ²Ð¾Ð¸ Ð´ÐµÐ½ÑÐ³Ð¸, 1000 Ð´Ð¾Ð»Ð»Ð°ÑÐ...",NaN,Nokia 8850
158911,/cellphones/samsung-sgh-600/,4/30/2015,tr,tr,Cepworld,cepworld.com,NaN,NaN,varsa satan 05312614607,muhsin,Samsung (936) SGH-600
160164,/cellphones/philips-genie-2000/,8/20/2000,it,it,Ciao,ciao.it,8.0,10.0,"Il Genie e' un buon telefono, scorrimente del ...",tonyblack,Philips Genie 2000


### Observations
1. Max_score for all the products is 10, i.e., none of the products has the max rating less than 10. Therefore there exists at-least 1 author for each product who provided the score as 10.
2. Datatype for `score` & `score_max` column are float. Remaining columns are of object datatype. This is expected, and there is no need to change the datatype of any column.
3. There exists 6 columns with no null values. Columns `score` and `score_max` have **63489** null values each; Column `extract` has **19361** null values; `author` has **63202** null values & column `product` has **1** null value. These null values need to be either removed or imputed. 
4. There are **6412** duplicate records out of **1415133** records. These duplicate rows need to be dropped.

## C. Round off scores to the nearest integers

In [17]:
## Since Score and Score_max columns have null values, we'll use "pd.Int64Dtype" to convert the float to integer
df_phone_review['score'].round(decimals=0).astype(pd.Int64Dtype())

0         10
1         10
2          6
3          9
4          4
          ..
163832     2
163833    10
163834     2
163835     8
163836     2
Name: score, Length: 1415133, dtype: Int64

In [18]:
df_phone_review['score_max'].round(decimals=0).astype(pd.Int64Dtype())

0         10
1         10
2         10
3         10
4         10
          ..
163832    10
163833    10
163834    10
163835    10
163836    10
Name: score_max, Length: 1415133, dtype: Int64

## D. Check for missing values. Impute the missing values, if any

In [19]:
print("Number of missing values for each column")
df_phone_review.isnull().sum()

Number of missing values for each column


phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [20]:
print("Percentage of missing values for each column")
df_phone_review.isnull().sum()*100/len(df_phone_review)

Percentage of missing values for each column


phone_url    0.000000
date         0.000000
lang         0.000000
country      0.000000
source       0.000000
domain       0.000000
score        4.486433
score_max    4.486433
extract      1.368140
author       4.466153
product      0.000071
dtype: float64

In [21]:
## Handling Missing Value for `product` column
df_phone_review[df_phone_review['product'].isnull()==True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
312960,/cellphones/samsung-galaxy-s-iii/,1/22/2014,de,de,Amazon,amazon.de,10.0,10.0,Bestes Smartphone was ich bisher hatte :) Ã¶af...,NaN,NaN


In [22]:
# Replacing nan product with empty string
df_phone_review['product'].fillna('Missing',inplace=True)

df_phone_review[df_phone_review['product'].isnull()==True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


In [23]:
## Handling Missing Value for `author` column
df_phone_review[df_phone_review['author'].isnull()==True]
df_phone_review['author'].fillna('Missing',inplace=True)

df_phone_review[df_phone_review['author'].isnull()==True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


In [24]:
## Handling Missing Value for `extract` column
df_phone_review[df_phone_review['extract'].isnull()==True]
df_phone_review['extract'].fillna('Missing',inplace=True)

df_phone_review[df_phone_review['extract'].isnull()==True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


In [25]:
## Handling missing values for score and score_max
df_phone_review['score'].fillna(int(df_phone_review['score'].mean()), inplace=True)
df_phone_review['score_max'].fillna(int(df_phone_review['score_max'].mean()), inplace=True)
print(df_phone_review[df_phone_review['score'].isnull()==True])
print(df_phone_review[df_phone_review['score_max'].isnull()==True])

Empty DataFrame
Columns: [phone_url, date, lang, country, source, domain, score, score_max, extract, author, product]
Index: []
Empty DataFrame
Columns: [phone_url, date, lang, country, source, domain, score, score_max, extract, author, product]
Index: []


## E. Check for duplicate values and remove them, if any

In [26]:
df_phone_review[df_phone_review.duplicated()==True].shape

(6414, 11)

In [27]:
df_phone_review.drop_duplicates(inplace=True)

In [28]:
df_phone_review[df_phone_review.duplicated()==True]

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


In [29]:
df_phone_review.shape

(1408719, 11)

- Hence, all the duplicate rows have been removed.

## F. Keep only 1 Million data samples. Use random state=612

In [30]:
df_phone_review.sample(n=1000000, random_state=612)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
164242,/cellphones/samsung-galaxy-a3-2016/,2/4/2016,de,de,Amazon,amazon.de,10.0,10.0,Also: ich habe das A3 2016 jetzt seit knapp ei...,Daniel,Samsung âSamsung Galaxy A3 Smartphone (12 cm...
100552,/cellphones/sony-xperia-z1-compact-d5503/,3/31/2014,es,es,Amazon,amazon.es,4.0,10.0,La pantalla estaba rayada y en el costado tamb...,Cliente Amazon,Sony Xperia Z1 Compact - Smartphone libre Andr...
277587,/cellphones/lenovo-zuk-z1/,12/21/2015,en,in,Amazon,amazon.in,6.0,10.0,Lots and lots of bugs ...,Amazon Customer,"Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)"
85949,/cellphones/apple-iphone-4/,6/14/2014,en,us,Amazon,amazon.com,10.0,10.0,I Bought it for me and I needed a new phone I ...,Chelsie Grainger,Verizon Apple iPhone 4 8GB No Contract 3G WiFi...
321346,/cellphones/asus-zenfone-2-laser-ze500kl/,2/15/2016,pt,br,Ofertou,shopping.ofertou.com,10.0,10.0,"O produto tem uma design bom, a curvatura tras...",e-bit,Asus ZenFone 2 ZE551ML 16GB Z3580
...,...,...,...,...,...,...,...,...,...,...,...
314560,/cellphones/lg-k7/,5/17/2016,en,us,Amazon,amazon.com,10.0,10.0,Pro easy to set up good pic love buttons on th...,Violet13,"LG K7 4G K330 LTE, Android, 8GB, No-Contract T..."
124447,/cellphones/nokia-5250/,10/3/2011,tr,tr,Tiklayaz,tiklayaz.com,8.0,10.0,iyatÄ±na ve karizmasÄ±na oranla daha ne istiyo...,huliye,Nokia 5250 Cep Telefonu
67070,/cellphones/huawei-p9-lite/,10/22/2016,it,it,Amazon,amazon.it,10.0,10.0,Smartphone con caratteristiche di fascia medio...,Giovanni Delogu,"Huawei P9 Lite Smartphone, LTE, Display 5,2â..."
308795,/cellphones/asus-zenfone-max-zc550kl/,4/11/2016,en,in,Amazon,amazon.in,6.0,10.0,Phone battery backup is good. However sound an...,Navin V.,"Asus Zenfone Max ZC550KL-6A068IN (Black, 2GB, ..."


## G. Drop irrelevant features. Keep features like Author, Product, and Score

In [31]:
df_phone_review.columns

Index(['phone_url', 'date', 'lang', 'country', 'source', 'domain', 'score',
       'score_max', 'extract', 'author', 'product'],
      dtype='object')

In [32]:
df_phone_review.drop(['phone_url', 'date','lang','source'], axis=1,inplace=True)

# Q2.

## A. Identify the most rated products

This question can be interpreted in 2 ways:
1. Products with most number of rating
2. Products with most ratings (i.e., highest scores)

Hence, I have shown both the interpretation below

In [33]:
# Interpretation 1
pd.DataFrame(df_phone_review.groupby(['product'])['score'].count().sort_values(ascending=False)).head(20)

,score
product,
"Lenovo Vibe K4 Note (White,16GB)",5223
"Lenovo Vibe K4 Note (Black, 16GB)",4389
"OnePlus 3 (Graphite, 64 GB)",4103
"OnePlus 3 (Soft Gold, 64 GB)",3558
Huawei P8lite zwart / 16 GB,2707
Samsung Galaxy Express I8730,2676
"Lenovo Vibe K5 (Gold, VoLTE update)",2534
Samsung Galaxy S6 zwart / 32 GB,2342
Nokia 5800 XpressMusic,2123


- The above table shows the top 20 products which have been rated the most. i.e., *Lenovo Vibe K4 Note (White,16GB)* has been rated 5223 times.

To show the products which have been given the highest ratings, we can use sum aggregate function instead of count function for "score" column

In [34]:
# Interpretation 2
pd.DataFrame(df_phone_review.groupby(['product'])['score'].sum().sort_values(ascending=False)).head(20)

,score
product,
"Lenovo Vibe K4 Note (White,16GB)",37502.0
"OnePlus 3 (Graphite, 64 GB)",35800.0
"Lenovo Vibe K4 Note (Black, 16GB)",31484.0
"OnePlus 3 (Soft Gold, 64 GB)",30252.0
Huawei P8lite zwart / 16 GB,22868.2
Samsung Galaxy Express I8730,22006.0
Samsung Galaxy S6 zwart / 32 GB,21065.6
Nokia 5800 XpressMusic,17466.3
"Lenovo Vibe K5 (Gold, VoLTE update)",17374.0


- Hence, the above table shows top 20 products with the highest products. Therefore, *Lenovo Vibe K4 Note (White,16GB)* has a total rating of 37502 which is the highest, followed by *OnePlus 3 (Graphite, 64 GB)* with 35800 score

## B. Identify the users with most number of reviews

In [35]:
pd.DataFrame(df_phone_review.groupby(['author'])['score'].count().sort_values(ascending=False)).head(20)

,score
author,
Amazon Customer,76933
Missing,61816
Cliente Amazon,19300
e-bit,8411
Client d'Amazon,7716
Amazon Kunde,4746
Anonymous,2749
einer Kundin,2610
einem Kunden,1898


- Hence, the above table shows the users (or authors) who have reviewed the most number of times. As you can see *Amazon Customer* has the most number of reviews.

## C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final datasetdf_phone_review

In [36]:
df_phone_review.shape

(1408719, 7)

In [37]:
df_phone_review['author'].value_counts()

Amazon Customer    76933
Missing            61816
Cliente Amazon     19300
e-bit               8411
Client d'Amazon     7716
                   ...  
monsieur x             1
Geens Yves             1
THD                    1
Guarowski              1
claudia0815            1
Name: author, Length: 801104, dtype: int64

In [38]:
df_phone_review['product'].value_counts()

Lenovo Vibe K4 Note (White,16GB)                                    5223
Lenovo Vibe K4 Note (Black, 16GB)                                   4389
OnePlus 3 (Graphite, 64 GB)                                         4103
OnePlus 3 (Soft Gold, 64 GB)                                        3558
Huawei P8lite zwart / 16 GB                                         2707
                                                                    ... 
Samsung i9100 Android SIM Free Smartphone                              1
SAMSUNG - Smartphone Galaxy A3 2017 bleu 16 Go                         1
Samsung Galaxy S2 T989 16GB Unlocked GSM Dual-Core Phone - White       1
Samsung SGH-D710                                                       1
Samsung S3 Mini i8190N 8GB Blue Unlocked                               1
Name: product, Length: 61314, dtype: int64

In [39]:
# Creating dataframe with product ratings more than 50
df_product_greater_than_50_temp = df_phone_review[df_phone_review['product'].map(df_phone_review['product'].value_counts() > 50)]
df_product_greater_than_50_temp

,country,domain,score,score_max,extract,author,product
0,us,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,us,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
4,us,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
5,us,verizonwireless.com,10.0,10.0,I am the type of person who never would comple...,BDB76,Samsung Galaxy S8
6,us,verizonwireless.com,4.0,10.0,The way this Samsung S8 phone operates is more...,KLC30306,Samsung Galaxy S8
...,...,...,...,...,...,...,...
163670,de,dooyoo.de,4.0,10.0,Also im groÃen und ganzen gesehen ist das GF7...,SBiernat,Ericsson GF768
163671,de,dooyoo.de,6.0,10.0,Das Ericsson 728 ist ensich schon ein gutes Ge...,Missing,Ericsson GF768
163672,de,dooyoo.de,6.0,10.0,Das Ericsson GF 768 ist ein sehr kleines und l...,MAD,Ericsson GF768
163673,de,dooyoo.de,4.0,10.0,Das Ericsson GF 768 wÃ¼rde ich als Siemens C25...,DukeMT,Ericsson GF768


In [40]:
# Creating dataframe with product ratings & author more than 50 
df_product_author_greater_than_50 = df_product_greater_than_50_temp[df_product_greater_than_50_temp['author'].map(df_product_greater_than_50_temp['author'].value_counts() > 50)]
df_product_author_greater_than_50

,country,domain,score,score_max,extract,author,product
259,nl,wehkamp.nl,10.0,10.0,heb deze telefoon nu een week heb geen spijt v...,schreef,Samsung Galaxy S8
271,ru,samsung.com,10.0,10.0,"ÐÑÐµÐ¼ ÑÐµÐºÐ¾Ð¼ÐµÐ½Ð´ÑÑ, Ð¾ÑÐµÐ½Ñ Ñ Ð¾...",ÐÐ³Ð¾Ñ,Samsung Galaxy S8
272,ru,samsung.com,10.0,10.0,"ÐÐ¿Ð¿Ð°ÑÐ°Ñ Ð¿Ð¾Ð½ÑÐ°Ð²Ð¸Ð»ÑÑ, Ð·Ð´Ð¾ÑÐ...",ÐÐ»Ð°Ð´Ð¸Ð¼Ð¸Ñ,Samsung Galaxy S8
273,ru,samsung.com,10.0,10.0,ÐÐ° Ð½ÐµÑÐºÐ¾Ð»ÑÐºÐ¾ Ð´Ð½ÐµÐ¹ Ð½ÐµÐ´Ð¾ÑÑÐ...,ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ,Samsung Galaxy S8
275,ru,samsung.com,8.0,10.0,ÐÐµ Ð´Ð»Ñ ÐºÐ¾Ð³Ð¾ Ð½Ðµ ÑÐµÐºÑÐµÑ ÑÑÐ¾ ...,Ð¡ÐµÑÐ³ÐµÐ¹,Samsung Galaxy S8
...,...,...,...,...,...,...,...
163454,de,dooyoo.de,8.0,10.0,Das Siemens S10 hat vor 2 Wochen ein Freund vo...,Missing,Siemens S10
163457,de,dooyoo.de,4.0,10.0,Das S10 ist sowohl vom Design als auch von der...,Missing,Siemens S10
163466,de,dooyoo.de,8.0,10.0,Das S10 war vor der EinfÃ¼hrung von Nokias Wun...,Missing,Siemens S10
163467,de,dooyoo.de,6.0,10.0,Starke Begleiter auf \n abenteuerlichen Wegen ...,Missing,Siemens S10


In [41]:
df_product_author_greater_than_50.shape

(202530, 7)

# Q3. Build a popularity based model and recommend top 5 mobile phones

In [42]:
df_product_author_greater_than_50.groupby('product')['score'].count().sort_values(ascending=False).head()  

product
Lenovo Vibe K4 Note (White,16GB)       3112
Lenovo Vibe K4 Note (Black, 16GB)      2575
OnePlus 3 (Graphite, 64 GB)            1876
OnePlus 3 (Soft Gold, 64 GB)           1784
Lenovo Vibe K5 (Gold, VoLTE update)    1592
Name: score, dtype: int64

In [43]:
mean_score_df = pd.DataFrame(df_product_author_greater_than_50.groupby('product')['score'].mean())

In [44]:
mean_score_df['number_of_ratings'] = pd.DataFrame(df_product_author_greater_than_50.groupby('product')['score'].count())  

In [45]:
mean_score_df

,score,number_of_ratings
product,,
3 FILMS PROTECTION ECRAN SAMSUNG Galaxy Ace S5830 [Appareils Ã©lectroniques],6.000000,2
5-Zoll- Android 4.2 Cubot P9 3G Smart Phone MTK6572 Dual Core 1.3GHz QHD IPS Schirm 512MB RAM 4GB ROM GPS 8MP...,10.000000,2
5.5-Inch Unlocked Lenovo A850 3G Smartphone-(960x540) Quad Core 4GB MT6582m 1331MHz Android 4.2 Dual Camera +Dual SIM -Black (Rooted + Google Play),5.714286,7
6682 / 6681,9.000000,2
"AICEK Coque ASUS ZenFone 2 ZE550ML/ZE551ML, AICEK Etui Silicone Gel ASUS ZenFone 2 Housse Antichoc ZenFone 2 Transparente Souple Coque de Protection pour ASUS ZenFone 2(5.5 Pouces)",9.586207,29
...,...,...
Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ Asus ZenFone 2,8.769231,13
×××¤×× ×¡××××¨× LG G4 H815,8.000000,8
×××¤×× ×¡××××¨× LG Nexus 5 16GB D821,8.857143,7


In [46]:
mean_score_df.sort_values(by=['score'], ascending=False).head()

,score,number_of_ratings
product,,
×××¤×× ×¡××××¨× Xiaomi Redmi Note 4 64GB,10.0,2
Blackberry Storm 9500 Smartphone (mit Branding Vodafone) schwarz,10.0,3
Doogee YOYAGER DG300 5.0 Inch QHD 3G Dual Core MTK6572W Dual SIM Dual Standby Smartphone Android 4.2 Mobile Phone GPS Cellphone WIFI,10.0,2
"Nokia Lumia 630 UK SIM-Free Smartphone - Green (Windows, 4.5-inch, 8GB)",10.0,6
"Nokia Lumia 930 Smartphone, 32 GB, Nero [Italia]",10.0,11


# Q4. Build a collaborative filtering model using SVD

In [47]:
df_phone_review_collab = df_product_author_greater_than_50[['author','product','score']]

In [48]:
from surprise import KNNWithMeans
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split

In [49]:
reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(df_phone_review_collab[['author', 'product', 'score']], reader)

In [50]:
trainset, testset = train_test_split(data, test_size=.15)

### Using SVD

In [52]:
from surprise import SVD, KNNWithMeans
from surprise import accuracy

In [53]:
svd_model = SVD(n_factors=5,biased=False, random_state=612)
svd_model.fit(trainset)

In [54]:
test_pred_svd = svd_model.test(testset)

### Using KNNWithMeans

In [56]:
algo_i = KNNWithMeans(k=10, sim_options={ 'user_based': False})

algo_i.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


# Q5. Evaluate the collaborative model. Print RMSE value

In [55]:
# compute RMSE
accuracy.rmse(test_pred_svd)

RMSE: 2.6338


2.6337807054072266

In [57]:
test_pred_knn = algo_i.test(testset)
print(accuracy.rmse(test_pred_knn))

RMSE: 2.6899
2.68993991988036


- Based on the above RMSE scores, it looks like for this case, SVD model is slightly better than KNNWithMeans model

## Q6. Predict score (average rating) for test users

In [69]:
test_pred_svd_df = pd.DataFrame(test_pred_svd)

In [75]:
test_pred_svd_df = test_pred_svd_df.rename({'est': 'score_pred'}, axis=1)

In [76]:
test_pred_svd_user_mean = pd.DataFrame(test_pred_svd_df.groupby('uid')['score_pred'].mean().sort_values(ascending=False))

In [77]:
test_pred_svd_user_mean

,score_pred
uid,
schreef,9.802794
einem Kunden,9.644456
ÐÐ»Ð°Ð´Ð¸ÑÐ»Ð°Ð²,9.581059
einer Kundin,9.547625
ÐÐ»ÐµÐºÑÐ°Ð½Ð´ÑÐ°,9.481827
...,...
Julie,6.615565
D,6.457841
Stephanie,6.341382


- The above table shows the average predicted scores of test users

## Q7. Report your findings and inferences

In [79]:
test_pred_svd_user_mean

,score_pred
uid,
schreef,9.802794
einem Kunden,9.644456
ÐÐ»Ð°Ð´Ð¸ÑÐ»Ð°Ð²,9.581059
einer Kundin,9.547625
ÐÐ»ÐµÐºÑÐ°Ð½Ð´ÑÐ°,9.481827
...,...
Julie,6.615565
D,6.457841
Stephanie,6.341382


In [80]:
test_pred_svd_user_mean.describe().T

,count,mean,std,min,25%,50%,75%,max
score_pred,533.0,8.134359,0.621872,2.442047,7.806053,8.132816,8.5284,9.802794


### Observations and Inference:
1. From the above data, we got 533 unique rows(or authors) with their average rating predicted.
2. The above table displays average rating for all the products given by a particular user.
2. The highest predicted score was for user *schreef* with **9.8** .
4. The overall mean predicted score for all users is 8.13.
5. The 75th percentile for the mean predicted score is 8.528400 and median score is 8.13. Hence, larger products lie with the rating of more than ~8.
3. The lowest predicted score was for user *Gast* with **2.44**.

# Q8. Try and recommend top 5 products for test users

In [141]:
## Displaying the prediction dataframe
test_pred_svd_df

,uid,iid,r_ui,score_pred,details
0,ÐÑÑÐµÐ¼,Sony Xperia Z1 Compact (Ð»Ð°Ð¹Ð¼),8.7,9.658144,{'was_impossible': False}
1,Anne,"Honor 4X Smartphone (5,5 Zoll (14 cm) Touch-Di...",10.0,9.957194,{'was_impossible': False}
2,ÐÐ¼Ð¸ÑÑÐ¸Ð¹,Sony Xperia Z (ÑÐµÑÐ½ÑÐ¹),10.0,8.304693,{'was_impossible': False}
3,Amazon Customer,LG Electronics Nexus 5X H791 32GB Carbon Black...,10.0,6.210250,{'was_impossible': False}
4,Stefan,Huawei P8 Champagne / 16 GB,8.2,7.911613,{'was_impossible': False}
...,...,...,...,...,...
30375,Missing,LG Nexus 5 16Gb D821,10.0,8.346849,{'was_impossible': False}
30376,MM,"LG Electronics E610 Optimus L5 Smartphone (10,...",8.0,6.264324,{'was_impossible': False}
30377,miatamania,Samsung F250 Cep Telefonu,8.0,7.981263,{'was_impossible': False}
30378,Amazon Customer,"VIVO V5 (Crown gold, 32 GB) (4 GB RAM)",2.0,6.937857,{'was_impossible': False}


In [143]:
test_pred_svd_group_sorted_df = test_pred_svd_df.groupby(["uid"]).apply(lambda x: x.sort_values(["score_pred"], ascending = False)).reset_index(drop=True)
# select top 5 products for each user
test_pred_svd_group_sorted_top5_df = pd.DataFrame(test_pred_svd_group_sorted_df.groupby('uid').head(5))

In [144]:
## Displaying top 5 products for each user
test_pred_svd_group_sorted_top5_df.head(20)

,uid,iid,r_ui,score_pred,details
0,#,Nokia 6680,10.0,9.417478,{'was_impossible': False}
1,#,Nokia N95,10.0,9.299457,{'was_impossible': False}
2,#,Nokia 6303i classic,6.0,9.272251,{'was_impossible': False}
3,#,Nokia 5230,8.0,9.115455,{'was_impossible': False}
4,#,Nokia 5230,8.0,9.115455,{'was_impossible': False}
26,????????,Sony Ericsson Xperia neo V MT11i,8.0,9.947987,{'was_impossible': False}
27,????????,Sony Ericsson Xperia neo V MT11i,10.0,9.947987,{'was_impossible': False}
28,????????,Samsung Galaxy S Advance,8.0,9.624258,{'was_impossible': False}
29,????????,Sony Xperia V (?????????????),7.3,9.464052,{'was_impossible': False}
30,????????,Sony Xperia V (?????????????),10.0,9.464052,{'was_impossible': False}


In [147]:
## Displaying top 5 products for **Amazon Customer** user
test_pred_svd_group_sorted_top5_df[test_pred_svd_group_sorted_top5_df['uid']=='Amazon Customer']

,uid,iid,r_ui,score_pred,details
664,Amazon Customer,Sprint Samsung Galaxy S5 White 16GB (Sprint Pr...,10.0,10.0,{'was_impossible': False}
665,Amazon Customer,Sprint Samsung Galaxy S5 White 16GB (Sprint Pr...,10.0,10.0,{'was_impossible': False}
666,Amazon Customer,Sprint Samsung Galaxy S5 White 16GB (Sprint Pr...,10.0,10.0,{'was_impossible': False}
667,Amazon Customer,Wileyfox Swift 4G Dual SIM-Free Smartphone - W...,10.0,10.0,{'was_impossible': False}
668,Amazon Customer,"Honor 4X Smartphone (5,5 Zoll (14 cm) Touch-Di...",10.0,10.0,{'was_impossible': False}


In [148]:
## Displaying top 5 products for **schreef** user
test_pred_svd_group_sorted_top5_df[test_pred_svd_group_sorted_top5_df['uid']=='schreef']

,uid,iid,r_ui,score_pred,details
28321,schreef,Samsung Galaxy A5 (2016),10.0,10.000000,{'was_impossible': False}
28322,schreef,Samsung Galaxy S7 edge,10.0,10.000000,{'was_impossible': False}
28323,schreef,Samsung Galaxy A5 (2016),10.0,10.000000,{'was_impossible': False}
28324,schreef,Samsung Galaxy S7 edge,10.0,10.000000,{'was_impossible': False}
28325,schreef,Samsung Galaxy J3 (2016),8.0,9.995947,{'was_impossible': False}


# Q9. Try other techniques (Example: cross validation) to get better results

In [149]:
from surprise.model_selection import cross_validate

In [150]:
results = cross_validate(svd_model, data, measures=['RMSE'], cv=10, verbose=True)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    2.6518  2.6292  2.6041  2.6301  2.6334  2.6325  2.6185  2.6098  2.6360  2.6239  2.6269  0.0130  
Fit time          1.55    1.61    1.63    1.51    1.67    1.62    1.66    1.61    1.49    1.49    1.59    0.07    
Test time         0.13    0.48    0.13    0.16    0.13    0.22    0.17    0.13    0.45    0.13    0.21    0.13    


In [151]:
print("Average RMSE: ", np.average(results["test_rmse"]))

Average RMSE:  2.6269237557120375


In [154]:
from surprise import SVD
from surprise.model_selection import GridSearchCV
 
param_grid = {
  'n_factors': [20, 50, 100],
  'n_epochs': [5, 10, 20]
}
 
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=10)
gs.fit(data)
 
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


2.5591023007142986
{'n_factors': 20, 'n_epochs': 5}


In [155]:
# best hyperparameters
best_factor = gs.best_params['rmse']['n_factors']
best_epoch = gs.best_params['rmse']['n_epochs']
trainset, testset = train_test_split(data, test_size=.20)
svd_tuned = SVD(n_factors=best_factor, n_epochs=best_epoch)
svd_tuned.fit(trainset)

In [156]:
test_pred_svd_tune = svd_tuned.test(testset)
accuracy.rmse(test_pred_svd_tune)

RMSE: 2.5636


2.5635616957439784

- Hence, using **Cross Validation** & **HyperParameter Tunning**, we improved the RMSE from 2.67 to 2.56

# Q10. In what business scenario you should use popularity based Recommendation Systems


- Popularity based Recommendation system makes use of current trend products. For instance, if a product is one that every new user typically purchases, there is a probability that it may recommend that product to the user who just signed up.
- Popularity based Recommendation Systems should be used when we don't have history about the user, but we still have to provide recommendation to the user, to provide a smooth experience, instead of just a blank site with no recommendation.
- It doesn't suffer from cold start issues, so it may make product recommendations for a variety of different filters from the very first day.

For E.g.,
1. Youtube/Instagram trending videos and reels, when the user first logs in
2. Spotify currently trending songs for a new user who just signed-up
3. Google News, filtered by most popular or trendy ones

# Q11. In what business scenario you should use CF based Recommendation Systems

- Collaborative Filtering works on the similarity across different users and also items that are widely used as an e-commerce or online movie websites, or songs app. It is type of an user item interactions
- CF Recommendation Systems will give more accurate results if we have large volume of data about the users and items.
- It doesn't mandatorily require attitional attributes for users and items to be known. 
- When we don't want to overspecialise user's profile, collaborative filtering is used to recommend items that are completely different from what they have seen before. For E.g., User who has watched Spiderman 1, should not only be recommended Spiderman 2,3, etc., but also other kinds of superhero movies.
- It can't be used to mitigate the cold start problem. CF requires history about user item interaction.

# Q12. What other possible methods can you think of which can further improve the recommendation for different users

1. Performing further hyperparameter tuning, and using various cross validation techniques, can help improve the rmse score.
2. Usually Collaborative filtering alone isn't sufficient. For E.g., it suffers from cold start problem. We can create Hybrod models, to better recommend the products to users
3. Using NLP and association rule mining, we can perform research like CountVec, NLTK on the extracts, or genre and group similar kinds together.
4. We can further use more features like demographics of the users, to better recommend products based on their country. For E.g., iPhone is majorly used in US but Xiami isn't.
5. Using different kinds of correlation measures like Pearson, Cosine and Mean Square Difference, assessing their results.
6. Getting to know the business or domain knowledge on the products, and more categories or features, can help further tweak and tunning the data. For E.g., camera quality, processor speed, etc. From user's perspective, having previous history of purchases like, is the user frugal in purchase, or able to purchases expensive products; Does the user have similar company's different products, like if a user already has an iwatch or an airpod, it is highly likely that he will purchase iphone.

=== THE END===